# SQL to circuit ansätze

This notebook implements the part of the algorithm which translates join order benchmark (JOB) and the simplified join order benchmark queries into pregroup diagrams and pregroup diagrams into circuit ansätze. The simple example can be found in `sql_to_circuit_simple_example` notebook.

The following code generates diagrams for all the SELECT-FROM-WHERE queries in the join order benchmark and their simplified versions. Running the code will take some time and it also works as a test package for the code. The diagrams are already generated in the folders `join-order-benchmark-diagrams` and `simplified-JOB-diagrams`.

Unfortunalyte, JOB queries produce too large circuits for quantum computing resources that we have available. That is why we created the set of simplified queries. As in the data generation case, this notebook is for reproducibility reasons and the user does not need to rerun this if they do not want to change the underlying queries or the mappings.

## Transformation 1: SQL to context-free grammar diagrams

In [1]:
from antlr4 import *
from SQLiteLexer import SQLiteLexer
from SQLiteParser import SQLiteParser
from SQLiteParserListener import SQLiteParserListener
import json
import os
import glob
from pathlib import Path
from discopy import Ty, Box, Functor
from functools import reduce
from discopy.utils import dumps, loads
this_folder = os.path.abspath(os.getcwd())

In [2]:
query_path_job = "\\join-order-benchmark-queries\\[0-9]*.sql"
query_path_training = "\\training_queries\\[0-9]*.sql"
query_path_test = "\\test_queries\\[0-9]*.sql"

In [3]:
def create_CFG_diagrams(query_path, output_folder_name):
    queries = glob.glob(this_folder + query_path)

    for count, query in enumerate(queries):
        print("Process: ", count, " out of ", len(queries))
        base_name = Path(query).stem
        try:
            input_stream = FileStream(query)
            lexer = SQLiteLexer(input_stream)
            stream = CommonTokenStream(lexer)
            parser = SQLiteParser(stream)
            tree = parser.parse()
            walker = ParseTreeWalker()
            listener = SQLiteParserListener(parser)
            walker.walk(listener, tree)
            diagram = listener.get_final_diagram().dagger()
            width = diagram.width()
            height = diagram.depth()
            dim = 3*max(width, height)
            diagram.draw(figsize=(dim, dim), path = this_folder + "\\" + output_folder_name + "\\" + base_name + ".png")
            with open(this_folder + "\\" + output_folder_name + "\\" + base_name + ".json", 'w') as outfile:
                json.dump(json.loads(dumps(diagram)), outfile)
        except:
            print("Query: ", base_name, " failed.")

In [4]:
#%%capture
create_CFG_diagrams(query_path_job, "join-order-benchmark-diagrams//cfg-diagrams")
create_CFG_diagrams(query_path_training, "simplified-JOB-diagrams//cfg-diagrams")
create_CFG_diagrams(query_path_test, "simplified-JOB-diagrams//cfg-diagrams")

Process:  0  out of  113
Process:  1  out of  113
Process:  2  out of  113
Process:  3  out of  113
Process:  4  out of  113
Process:  5  out of  113
Process:  6  out of  113
Process:  7  out of  113
Process:  8  out of  113
Process:  9  out of  113
Process:  10  out of  113
Process:  11  out of  113
Process:  12  out of  113
Process:  13  out of  113
Process:  14  out of  113
Process:  15  out of  113
Process:  16  out of  113
Process:  17  out of  113
Process:  18  out of  113
Process:  19  out of  113
Process:  20  out of  113
Process:  21  out of  113
Process:  22  out of  113
Process:  23  out of  113
Process:  24  out of  113
Process:  25  out of  113
Process:  26  out of  113
Process:  27  out of  113
Process:  28  out of  113
Process:  29  out of  113
Process:  30  out of  113
Process:  31  out of  113
Process:  32  out of  113
Process:  33  out of  113
Process:  34  out of  113
Process:  35  out of  113
Process:  36  out of  113
Process:  37  out of  113
Process:  38  out of  

## Transformation 2: Context-free grammar diagrams to pregroup grammar diagrams

In [7]:
from pregroup_functor_mappings import count_boxes, object_mapping, arrow_mapping

def create_pregroup_grammar_diagrams(cfg_folder_name, pregroup_folder_name):
    cfg_diagrams = glob.glob(this_folder + "\\" + cfg_folder_name + "\\[0-9]*.json")
    
    for count, serialized_diagram in enumerate(cfg_diagrams):
        print("Process: ", count, " out of ", len(cfg_diagrams))
        base_name = Path(serialized_diagram).stem
        f = open(serialized_diagram, "r")
        data = f.read()
        diagram = loads(data)
        
        num_of_result_columns = count_boxes(diagram, "result-column")
        num_of_result_columns += count_boxes(diagram, "result-column-with-alias")
        num_of_tables = count_boxes(diagram, "table")
        num_of_tables += count_boxes(diagram, "table-with-alias")
        
        Rewriter = Functor(ob = lambda x: object_mapping(x, num_of_result_columns, num_of_tables), ar = lambda f: arrow_mapping(f, num_of_result_columns, num_of_tables))
        
        try:
            pregroup_diagram = Rewriter(diagram)
            width = diagram.width()
            height = diagram.depth()
            dim = 3*max(width, height)
            pregroup_diagram.draw(figsize=(dim, dim), path = this_folder + "\\" + pregroup_folder_name + "\\" + base_name + ".png")

            with open(this_folder + "\\" + pregroup_folder_name + "\\" + base_name + ".json", 'w') as outfile:
                    json.dump(json.loads(dumps(pregroup_diagram)), outfile)
        except:
            print("Query: ", base_name, " failed.")

In [8]:
create_pregroup_grammar_diagrams("join-order-benchmark-diagrams//cfg-diagrams", "join-order-benchmark-diagrams//pregroup-diagrams")
create_pregroup_grammar_diagrams("simplified-JOB-diagrams//cfg-diagrams", "simplified-JOB-diagrams//pregroup-diagrams")
create_pregroup_grammar_diagrams("simplified-JOB-diagrams//cfg-diagrams", "simplified-JOB-diagrams//pregroup-diagrams")

Process:  0  out of  113
Process:  1  out of  113
Process:  2  out of  113
Process:  3  out of  113
Query:  11a  failed.
Process:  4  out of  113
Query:  11b  failed.
Process:  5  out of  113
Query:  11c  failed.
Process:  6  out of  113
Query:  11d  failed.
Process:  7  out of  113
Query:  12a  failed.
Process:  8  out of  113
Query:  12b  failed.
Process:  9  out of  113
Query:  12c  failed.
Process:  10  out of  113
Process:  11  out of  113
Query:  13b  failed.
Process:  12  out of  113
Query:  13c  failed.
Process:  13  out of  113
Process:  14  out of  113
Query:  14a  failed.
Process:  15  out of  113
Query:  14b  failed.
Process:  16  out of  113
Query:  14c  failed.
Process:  17  out of  113
Process:  18  out of  113
Process:  19  out of  113
Query:  15c  failed.
Process:  20  out of  113
Process:  21  out of  113
Process:  22  out of  113
Process:  23  out of  113
Process:  24  out of  113
Process:  25  out of  113
Process:  26  out of  113
Process:  27  out of  113
Process: 

## Transformation 3: pregroup diagram rewriting, cup removal and simplification

In [ ]:
def cup_remove_arrow_mapping(box):
    if box.name.lower() == 'select':
        return box
    elif not box.cod:
        domain = box.dom
        raised_leg = Ty(domain[0])
        new_domain = reduce(lambda x, y : x @ Ty(y), domain[1:], Ty())
        new_box = Id(raised_leg) @ Box(box.name, new_domain, raised_leg.l)\
        >> Cup(raised_leg, raised_leg.l)
        return new_box
    return box

def cup_remove_arrow_mapping2(box):
    if box.cod == box.dom == Ty('n'):
        return Id(box.cod)
    return box

cup_removal_functor = Functor(ob = lambda x: x, ar = lambda f: cup_remove_arrow_mapping(f))
cup_removal_functor2 = Functor(ob = lambda x: x, ar = lambda f: cup_remove_arrow_mapping2(f))

In [ ]:
def remove_cups_and_simplify(pregroup_folder_name, cup_removed_pregroup_folder_name):
    pregroup_diagrams = glob.glob(this_folder + "\\" + pregroup_folder_name + "\\[0-9]*.json")
    
    cupless_pregroup_diagram = cup_removal_functor(pregroup_diagram.normal_form()).normal_form()
    cupless_pregroup_diagram = cup_removal_functor2(cupless_pregroup_diagram).normal_form()
    width = cupless_pregroup_diagram.width()
    height = cupless_pregroup_diagram.depth()
    dim = 3*max(width, height)
    cupless_pregroup_diagram.draw(figsize=(dim, dim))

## Transformation 4: pregroup diagrams to circuit ansätze

In [ ]:
n, s = Ty('n'), Ty('s')
ansatz = IQPAnsatz({n: 1, s: 2}, n_layers=1, n_single_qubit_params=3)
    
def create_circuit_ansatz():
    
    circuit_diagram = ansatz(cupless_pregroup_diagram)
    width = circuit_diagram.width()
    height = circuit_diagram.depth()
    dim = 3*max(width, height)
    circuit_diagram.draw(figsize=(dim, dim)) #, path = this_folder + "\\figures\\circuit_figure.png")